In [1]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

# This code Scraps a single query from Amazon | Purpose modifed

from bs4 import BeautifulSoup as soup
import requests
import os
import csv

# vars with GLOBAL scope
bot_query = "https://www.amazon.in/s?k="  # Defualt order, no filters are being applied
# HEADER = {
#     "User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7"
# }
HEADER={"User-Agent": "Python-urlli~"}
PROXY = {
    "https": "138.0.207.18:58566"
}  # {"https": "https//124.107.229.210:8080", "http":"http//124.107.229.210:8080"} | Proxy Rotation is absent | pip package will be released sooner
csv_index = 0  # To keep propering indexing when adding data from different pages

## Error handling needed when image sizes mismatch
def csv_append(
    user_query="-",
    item_count="-",
    name="-",
    item_page="-",
    rating="-",
    review="-",
    is1="-",
    is15="-",
    is2="-",
    is25="-",
    is3="-",
    price="-",
):
    global is_first
    global csv_index

    csv_index += 1
    item_count = csv_index  # Overtake
    csv_name = user_query.replace("+", "-") + ".csv"
    if is_first:
        with open(csv_name, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(
                [
                    "SN",
                    "Name",
                    "Link to item",
                    "Rating",
                    "Reviews",
                    "Image Download link 1x",
                    "Image Download link 1.5x",
                    "Image Download link 2x",
                    "Image Download link 2.5x",
                    "Image Download link 3x",
                    "Price",
                ]
            )
            writer.writerow(
                [
                    item_count,
                    name,
                    item_page,
                    rating,
                    review.replace(",", ""),
                    is1,
                    is15,
                    is2,
                    is25,
                    is3,
                    price,
                ]
            )
    else:
        with open(csv_name, "a+", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(
                [
                    item_count,
                    name,
                    item_page,
                    rating,
                    review.replace(",", ""),
                    is1,
                    is15,
                    is2,
                    is25,
                    is3,
                    price,
                ]
            )


def slugify(img_name):
    """
    Slug Generator with image format
    :param1: image-name
    :return: slugged and format typed image
    """

    sluged_name = (
        img_name.replace(" ", "-").replace('"', "").replace("(", " ").replace(")", " ")
    )
    sluged_name = (
        sluged_name.replace("|", " ").replace("&", " ").replace(":", " ")
    )  # ASCII Range can be checked instead, More info needed about how will the images \
    # be used.

    return "{}.{}".format(sluged_name, "jpg")


def img_download(url, img_name):  # img_name):
    """
    Downloads the image from given url
    :param1: url
    :param2: imgage-name
    """
    # yes change here
    global csv_index

    # img_name = slugify(img_name).encode('ascii', 'ignore')
    img_name = slugify(str(csv_index))
    img = requests.get(url, headers=HEADER)
    with open(img_name, "wb") as f:  # Put them in folder properly
        f.write(img.content)


def scrape_bundle(user_query, page_count):

    global is_first
    is_first = True
    for count in range(1, int(page_count) + 1):
        scrape_query(user_query, count)
        print(count)
        is_first = False


def scrape_query(user_query, page_count):
    """
    Scrape and store the details of first page in csv that is when we search an item on amazon. Image is also downloaded.
    :param1: Name of item
    :return: None if no item appears
    Scraped Data:: Link to item | Item Name | Review count | Item Rating | Image download links | Item Availability | Price
    """

    global is_first

    amazon_query = bot_query + user_query + "&" + "page=" + str(page_count)

    client = requests.get(amazon_query, headers=HEADER)  # proxies=PROXY | gimmeproxy
    page_html = client.text

    page_soup = soup(page_html, "html.parser")
    container = page_soup.findAll("div", {"class": "s-result-item"})

    bundled_data = []

    ## Function can be made for below code | too much arg passing
    ## Better Error Handling is needed | can be avoided by key check
    # Taking only first item | Run complete loop to get all single page results
    if len(container) > 0:
        item_count = 0
        for item in container:
            try:

                item_count += 1
                item_page = "https://www.amazon.in" + item.a["href"]
                print(item_page)

                try:
                    image_srcset = [
                        (img_link.split(" ")[0], img_link.split(" ")[1])
                        for img_link in item.a.img["srcset"].split(", ")
                    ]
                except:
                    image_srcset = None
                try:
                    img_alt = item.img["alt"]  # item-name
                except:
                    img_alt = ""
                try:
                    rating = item.span.find("span", {"class": "a-icon-alt"}).text
                    review_count = item.span.find("span", {"class": "a-size-base"}).text
                except:
                    rating = "-"
                    review_count = "-"

                # Availability
                for x in item.span.find_all("span"):
                    if x.text == "Currently unavailable.":
                        availability = "Currently unavailable."
                else:
                    availability = "Available"

                price = item.span.find("span", {"class": "a-price-whole"})
                if price != None:
                    price = price.text
                else:
                    price = "-"

                # Downloading most HR image
                try:
                    img_download(url=image_srcset[-1][0], img_name=img_alt)
                except Exception as e:
                    print(str(e))

                try:
                    is1 = image_srcset[0][0]
                    is15 = image_srcset[1][0]
                    is2 = image_srcset[2][0]
                    is25 = image_srcset[3][0]
                except:
                    is1 = "-"
                    is15 = "-"
                    is2 = "-"
                    is25 = "-"
                try:  # Maybe 1 Image we have
                    is3 = image_srcset[-1][0]
                except:
                    is3 = "-"

                csv_append(
                    user_query=user_query,
                    item_count=item_count,
                    name=img_alt,
                    item_page=item_page,
                    rating=rating,
                    review=review_count,
                    is1=is1,
                    is15=is15,
                    is2=is2,
                    is25=is25,
                    is3=is3,
                    price=price,
                )
                is_first = False
            except:
                pass
    else:
        item = None
        return item


if __name__ == "__main__":

    user_query = input("Enter your Product: ").replace(
        " ", "+"
    )  # More filtering is needed | search 'mobile' to get all mobile data scraped
    page_count = input("Scrape upto how many pages: ")

    if page_count in ["0", " ", None]:
        scrape_bundle(user_query, "1")
    else:
        scrape_bundle(user_query, page_count)

Enter your Product: women tshirt
Scrape upto how many pages: 5
https://www.amazon.in/gp/bestsellers/apparel/1968545031/ref=sr_bs_0_1968545031_1
'NoneType' object is not subscriptable
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A00325423PJNJXDTKVMID&url=%2FSymbol-Womens-Regular-T-Shirt-RN-PO3-COMBO25_White%2Fdp%2FB07JQK21X3%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dwomen%2Btshirt%26qid%3D1596946782%26sr%3D8-2-spons%26psc%3D1&qualifier=1596946781&id=309829317023189&widgetName=sp_atf
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0649083HUK7Z0H8PVWQ&url=%2FPlush-Womens-Regular-Tshirts-M-08-PNK%2Fdp%2FB08CZ372CP%2Fref%3Dsr_1_3_sspa%3Fdchild%3D1%26keywords%3Dwomen%2Btshirt%26qid%3D1596946782%26sr%3D8-3-spons%26psc%3D1&qualifier=1596946781&id=309829317023189&widgetName=sp_atf
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A09586533NBH13

https://www.amazon.in/Neysa-T-Shirt-WomenCombo-01_Black-Red_Large/dp/B0841MMYHN/ref=sr_1_33?dchild=1&keywords=women+tshirt&qid=1596946782&sr=8-33
https://www.amazon.in/Long-T-Shirts-Women-Black/dp/B07VLFJPRM/ref=sr_1_34?dchild=1&keywords=women+tshirt&qid=1596946782&sr=8-34
https://www.amazon.in/Broadstar-Women-Cotton-Mustard-T-Shirt-S/dp/B07TLC416N/ref=sr_1_35?dchild=1&keywords=women+tshirt&qid=1596946782&sr=8-35
https://www.amazon.in/Quote-Marshals-Expecto-Petronum-T-Shirt/dp/B0872LFLZH/ref=sr_1_36?dchild=1&keywords=women+tshirt&qid=1596946782&sr=8-36
https://www.amazon.in/gp/video/ssoredirect/?ru=https%3A%2F%2Fapp.primevideo.com%2Fdetail%3Fgti%3Damzn1.dv.gti.feae84e9-cd86-25c8-2777-ebabb0f4e5a3%26ref_%3Ddvm_src_ret_in_xx_s&page-type-id=B07MFCG9WS&ref=sr_1_37&keywords=women+tshirt&qid=1596946782&sr=8-37
https://www.amazon.in/T2F-Regular-T-Shirt-wom-solid-08_-X-Large/dp/B08BY8Q4H5/ref=sr_1_38?dchild=1&keywords=women+tshirt&qid=1596946782&sr=8-38
https://www.amazon.in/MIDAAS-Womens-Cott

https://www.amazon.in/Neysa-Womens-T-Shirt-WomenCombo-01_Multicolored_X-Large/dp/B0841MLQBF/ref=sr_1_65?dchild=1&keywords=women+tshirt&qid=1596946820&sr=8-65
https://www.amazon.in/RSquarre-Womens-Printed-Friends-T-Shirt/dp/B06XD1VRC5/ref=sr_1_66?dchild=1&keywords=women+tshirt&qid=1596946820&sr=8-66
https://www.amazon.in/Veirdo-Womens-T-Shirt-GL_02_CAMO_M_Multicolored_Medium/dp/B083DXHK86/ref=sr_1_67?dchild=1&keywords=women+tshirt&qid=1596946820&sr=8-67
https://www.amazon.in/Rupa-Softline-Round-Black-X-Large/dp/B07FTFMVV6/ref=sr_1_68?dchild=1&keywords=women+tshirt&qid=1596946820&sr=8-68
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_mtf_aps_sr_pg2_1?ie=UTF8&adId=A01566632Z8ZGPRD9Q6E3&url=%2FFoxrobe-Womens-Cotton-LiveinMoment-Round%2Fdp%2FB0851GX421%2Fref%3Dsr_1_69_sspa%3Fdchild%3D1%26keywords%3Dwomen%2Btshirt%26qid%3D1596946820%26sr%3D8-69-spons%26psc%3D1&qualifier=1596946820&id=8942103564802205&widgetName=sp_mtf
https://www.amazon.in/gp/slredirect/picassoRedirect.ht

https://www.amazon.in/gp/bestsellers/apparel/1968545031/ref=sr_bs_0_1968545031_1
'NoneType' object is not subscriptable
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg3_1?ie=UTF8&adId=A002657030G7O3MYU568T&url=%2FSymbol-Womens-Regular-T-Shirt-RN-PO3-COMBO23_Black%2Fdp%2FB07JQ4D97H%2Fref%3Dsr_1_98_sspa%3Fdchild%3D1%26keywords%3Dwomen%2Btshirt%26qid%3D1596946848%26sr%3D8-98-spons%26psc%3D1&qualifier=1596946848&id=3371461838815767&widgetName=sp_atf_next
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg3_1?ie=UTF8&adId=A0309602R1Y0I0Z0BKCU&url=%2FVan-Heusen-Womens-55402_Pink-Blush_M%2Fdp%2FB07JVPNBFV%2Fref%3Dsr_1_99_sspa%3Fdchild%3D1%26keywords%3Dwomen%2Btshirt%26qid%3D1596946848%26sr%3D8-99-spons%26psc%3D1&qualifier=1596946848&id=3371461838815767&widgetName=sp_atf_next
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg3_1?ie=UTF8&adId=A0649083HUK7Z0H8PVWQ&url=%2FPlush-Womens-Regula

https://www.amazon.in/Wrangler-Womens-Regular-T-Shirt-W3347923312Z_Jsw-White/dp/B07XQM8QKJ/ref=sr_1_143?dchild=1&keywords=women+tshirt&qid=1596946848&sr=8-143
https://www.amazon.in/Mirako-Van-Heusen-Regular-VWCTURGBC09690_/dp/B08D8HN8Q9/ref=sr_1_144?dchild=1&keywords=women+tshirt&qid=1596946848&sr=8-144
https://www.amazon.in/DJ-fbb-Striped-Regular-1000865825036_Red_XXL/dp/B0794QP38P/ref=sr_1_145?dchild=1&keywords=women+tshirt&qid=1596946848&sr=8-145
https://www.amazon.in/VERO-MODA-Regular-T-Shirt-2052301003_Misty/dp/B07N49S2FF/ref=sr_1_146?dchild=1&keywords=women+tshirt&qid=1596946848&sr=8-146
https://www.amazon.in/ONLY-Women-Casual-_5713728523557_Dark-Melange_Medium_/dp/B07DDD2Q9F/ref=sr_1_147?dchild=1&keywords=women+tshirt&qid=1596946848&sr=8-147
https://www.amazon.in/MIDAAS-Womens-Cotton-Tshirts/dp/B079WTJ4S4/ref=sr_1_148?dchild=1&keywords=women+tshirt&qid=1596946848&sr=8-148
https://www.amazon.in/METRO-FASHION-Color-Block-Women-T-Shirt/dp/B07X8XNS2V/ref=sr_1_149?dchild=1&keywords=w

https://www.amazon.in/Miss-Chase-Regular-T-Shirt-MCS18TS01-17-82_White_Medium/dp/B079Z5CCLV/ref=sr_1_170?dchild=1&keywords=women+tshirt&qid=1596946880&sr=8-170
https://www.amazon.in/Jockey-Regular-T-Shirt-AW15_Banana-Melange_Large/dp/B07TBN589S/ref=sr_1_171?dchild=1&keywords=women+tshirt&qid=1596946880&sr=8-171
https://www.amazon.in/TRAZO-Womens-Cotton-Printed-T-Shirt/dp/B07D16BTT6/ref=sr_1_172?dchild=1&keywords=women+tshirt&qid=1596946880&sr=8-172
https://www.amazon.in/Modeve-Printed-T-Shirts-Women-XX-Large/dp/B07GLLWL1L/ref=sr_1_173?dchild=1&keywords=women+tshirt&qid=1596946880&sr=8-173
https://www.amazon.in/Fabricorn-Womens-T-Shirt-GT04_Grn_BlkWhtBlkRib_M_Green_Medium/dp/B0832NZ1W8/ref=sr_1_174?dchild=1&keywords=women+tshirt&qid=1596946880&sr=8-174
https://www.amazon.in/Jockey-Womens-T-Shirt-8901326163191_1515_Medium_Pink-Melange/dp/B071LG3NWL/ref=sr_1_175?dchild=1&keywords=women+tshirt&qid=1596946880&sr=8-175
https://www.amazon.in/Appulse-Womens-Sleeve-Thumbopen-Tshirt/dp/B07GFJKD4

https://www.amazon.in/FreshTrend-Womens-Cotton-T-shirt-Medium/dp/B07J5VG5MR/ref=sr_1_202?dchild=1&keywords=women+tshirt&qid=1596946913&sr=8-202
https://www.amazon.in/Womens-regular-printed-T-Shirt-SU20YCP05PINK_Pink_Small/dp/B0873RF97Q/ref=sr_1_203?dchild=1&keywords=women+tshirt&qid=1596946913&sr=8-203
https://www.amazon.in/LAYA-Printed-T-Shirt-X-Large-Multicolored/dp/B08C27K9N1/ref=sr_1_204?dchild=1&keywords=women+tshirt&qid=1596946913&sr=8-204
https://www.amazon.in/VERO-MODA-Abstract-10179746_Snow-White_Small/dp/B01NGYEX8I/ref=sr_1_205?dchild=1&keywords=women+tshirt&qid=1596946913&sr=8-205
https://www.amazon.in/Wrangler-Regular-T-Shirt-W309281048YZ_Jsw-Navy_Medium/dp/B07MLHNT81/ref=sr_1_206?dchild=1&keywords=women+tshirt&qid=1596946913&sr=8-206
https://www.amazon.in/Broadstar-Women-Cotton-Round-T-Shirt/dp/B07QBP5Y2M/ref=sr_1_207?dchild=1&keywords=women+tshirt&qid=1596946913&sr=8-207
https://www.amazon.in/Levis-Womens-Regular-T-Shirt-80829-0005_White/dp/B07X8XFVWC/ref=sr_1_208?dchild=

https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg5_1?ie=UTF8&adId=A07027851DFJFXXWM2OPV&url=%2FSHAUN-Womens-Printed-T-Shirt-105wwn3_npa38_Multicolor_Small%2Fdp%2FB07STWLTF1%2Fref%3Dsr_1_250_sspa%3Fdchild%3D1%26keywords%3Dwomen%2Btshirt%26qid%3D1596946913%26sr%3D8-250-spons%26psc%3D1&qualifier=1596946912&id=6876003081261660&widgetName=sp_btf
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg5_1?ie=UTF8&adId=A082473111RIBWXTOAD8V&url=%2FONLY-Women-Regular-T-Shirt-15149950_Chrysanthemum_Small%2Fdp%2FB0753NN71B%2Fref%3Dsr_1_251_sspa%3Fdchild%3D1%26keywords%3Dwomen%2Btshirt%26qid%3D1596946913%26sr%3D8-251-spons%26psc%3D1&qualifier=1596946912&id=6876003081261660&widgetName=sp_btf
https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg5_1?ie=UTF8&adId=A05839496T47W6H1GMEO&url=%2FVERO-MODA-Regular-T-Shirt-10236094_%2Fdp%2FB084W1WQ8V%2Fref%3Dsr_1_252_sspa%3Fdchild%3D1%26keywords%3Dwomen%2Btshirt%26qid%3D159694